# Vanilla Incremental Elicitation Procedure

In [1]:
import numpy as np
from mip import Model, xsum, BINARY, minimize, maximize

In [2]:
# Fonction pour calculer le modèle de décision avec les poids
def M_omega(x, omega):
    return np.dot(x, omega)

- Calcule l'évaluation d'une solution 'x' avec un vecteur de poids 'omega'.
    
    Args:
    - x (array-like): Vecteur représentant une solution avec ses scores pour chaque critère.
    - omega (array-like): Vecteur représentant les poids associés à chaque critère.
    
    Returns:
    - float: La somme pondérée des scores de 'x' selon les poids 'omega'.

In [3]:
# Fonction pour poser une question au décideur
def Query(x, y):
    # Simule la préférence du décideur (choix aléatoire ici, à remplacer par une vraie préférence)
    print(f"Comparaison entre {x} et {y}")
    return x if np.random.rand() > 0.5 else y

 - Pose une question au décideur pour comparer deux solutions 'x' et 'y'.
    La réponse est simulée par un choix aléatoire.
    
    Args:
    - x (array-like): Première solution à comparer.
    - y (array-like): Seconde solution à comparer.
    
    Returns:
    - array-like: La solution préférée entre 'x' et 'y'.

#### Résolution du probléme linéaire pour calculer PMR en utilisant MIP

La fonction PMR calcule le regret maximum par paires entre deux solutions x et y, en tenant compte des poids admissibles définis dans 
Ω. Le regret est la différence entre les évaluations des deux solutions, maximisée pour le pire ensemble de poids possible dans le polytope Ω. 

- Modélisation du problème : On modélise cela comme un problème de programmation linéaire où on veut maximiser la différence entre l’évaluation de y et de x en fonction des poids w (qui sont les variables de décision dans le modèle).

- On introduit une variable w pour chaque critère (chaque dimension du problème). Ces variables représentent les poids wi associés à chaque critère, qui doivent être trouvés pour maximiser le regret. Ces poids doivent respecter des contraintes de normalisation (somme égale à 1) et les autres contraintes définies dans Ω.

- Ajout des contraintes de Ω: Ω est un polytope défini par un ensemble de contraintes linéaires. Chaque contrainte dans Ω a la forme d’une équation linéaire qui doit être respectée pour que les poids soient admissibles. Dans le code, on itère sur toutes les contraintes définies dans Ω et on les ajoute au modèle.

-L’objectif est de maximiser la différence entre l'évaluation de y et x, c'est-à-dire maximiser Mw(y)-Mw(x), où Mw(.) est l’évaluation pondérée d’une solution par rapport aux poids w. Cela revient à maximiser la somme pondérée des différences entre y[i] et x[i].

In [4]:
# Fonction pour calculer le regret maximum par paires en tenant compte des contraintes de Omega
def PMR(x, y, Omega):
    # Créer un modèle mip pour résoudre le problème linéaire
    m = Model(sense=maximize)
    
    m_crit = len(x)  # Nombre de critères
    
    # Variables de décision : poids w_i pour chaque critère
    w = [m.add_var(lb=0) for _ in range(m_crit)]
    
    # Contrainte : somme des poids w_i = 1 (normalisation des poids)
    m += xsum(w[i] for i in range(m_crit)) == 1
    
    # Contrainte : ajouter les contraintes définies par le polytope Omega
    for constraint in Omega:
        m += xsum(w[i] * constraint['coefficients'][i] for i in range(m_crit)) >= constraint['rhs'] #la somme pondérée des variables 
                                                                                                     #de décision wi
    
    # Objectif : maximiser M_omega(y) - M_omega(x)
    m.objective = xsum(w[i] * (y[i] - x[i]) for i in range(m_crit))
    
    # Résoudre le problème
    m.optimize()
    
    # Obtenir le regret maximal
    return m.objective_value

 - Calcule le regret maximum par paires (PMR) entre deux solutions 'x' et 'y'.
    Utilise la programmation linéaire pour maximiser le regret sous les contraintes définies dans 'Omega'.
    
    Args:
    - x (array-like): Première solution.
    - y (array-like): Seconde solution.
    - Omega (list): Polytope des poids admissibles défini par un ensemble de contraintes.
    
    Returns:
    - float: La valeur du regret maximum pour la paire (x, y).

### Fonction Update : Mise à jour du polytope Ω

La fonction Update a pour but de mettre à jour le polytope Ω, qui représente l'ensemble des poids possibles, en fonction des réponses du décideur à une question de préférence entre deux solutions x et y.

- Décision du décideur : Le décideur est interrogé pour savoir s'il préfère x ou y. En fonction de sa réponse, on sait que le modèle de décision sous-jacent doit respecter certaines relations linéaires entre les poids w:
    * Si x est préféré à y, cela signifie que la somme pondérée des critères pour x doit être plus grande ou égale à celle de y:         Mw(x) >= Mw(y).
    * Sinon, l'inverse.

- Ici, les coefficients de la contrainte sont la différence entre les vecteurs x et y. Le côté droit de l'inégalité est 0 car on compare directement les évaluations pondérées.

- Mise à jour du polytope Ω : Une nouvelle contrainte est ajoutée au polytope Ω à chaque nouvelle réponse. Cette contrainte réduit l'espace des poids admissibles en excluant ceux qui ne respectent pas la préférence du décideur. Cette mise à jour affine la connaissance des préférences du décideur.

In [5]:
# Fonction pour mettre à jour le polytope des poids possibles Omega
def Update(Omega, answer, x, y):
    new_constraint = {}
    if np.array_equal(answer, x):
        # x est préféré à y -> ajouter la contrainte M_omega(x) >= M_omega(y)
        new_constraint['coefficients'] = x - y
        new_constraint['rhs'] = 0
    else:
        # y est préféré à x -> ajouter la contrainte M_omega(y) >= M_omega(x)
        new_constraint['coefficients'] = y - x
        new_constraint['rhs'] = 0
    
    # Mettre à jour le polytope Omega en ajoutant la nouvelle contrainte
    Omega.append(new_constraint)
    
    return Omega

 - Met à jour le polytope des poids admissibles 'Omega' en ajoutant une nouvelle contrainte
    basée sur la réponse du décideur. Si 'x' est préféré à 'y', on ajoute la contrainte M_omega(x) >= M_omega(y).
    
    Args:
    - Omega (list): Polytope des poids admissibles représenté sous forme de contraintes.
    - answer (array-like): La solution préférée (soit 'x' soit 'y').
    - x (array-like): Première solution.
    - y (array-like): Seconde solution.
    
    Returns:
    - list: Le polytope 'Omega' mis à jour avec la nouvelle contrainte.

In [6]:
# Fonction pour calculer le regret maximum (MR)
def MR(x, X, Omega):
    return max(PMR(x, y, Omega) for y in X if not np.array_equal(x, y))

- Calcule le regret maximum d'une solution 'x' par rapport à toutes les autres solutions dans 'X'.
    
    Args:
    - x (array-like): Solution pour laquelle on calcule le regret.
    - X (array-like): Ensemble des solutions.
    - Omega (list): Polytope des poids admissibles.
    
    Returns:
    - float: Le regret maximum pour la solution 'x'.

In [7]:
# Fonction pour calculer le regret minimax (mMR) et obtenir les solutions x_star et y_star
def mMR(X, Omega):
    x_star = min(X, key=lambda x: MR(x, X, Omega))
    y_star = max(X, key=lambda y: PMR(x_star, y, Omega))
    max_regret = MR(x_star, X, Omega)
    return max_regret, x_star, y_star

- Calcule le regret minimax parmi un ensemble de solutions 'X'.
    Trouve la solution 'x_star' qui minimise le regret maximum et la solution 'y_star' qui maximise le regret.
    
    Args:
    - X (array-like): Ensemble des solutions à comparer.
    - Omega (list): Polytope des poids admissibles défini par des contraintes.
    
    Returns:
    - tuple: (regret_minimax, x_star, y_star) où:
        - regret_minimax (float): La valeur du regret minimax.
        - x_star (array-like): La solution minimisant le regret.
        - y_star (array-like): La solution maximisant le regret par rapport à 'x_star'.

In [8]:
# Algorithme d'élucidation incrémentale vanilla
def vanilla_incremental_elicitation(X, Omega, epsilon):
    max_regret, x_star, y_star = mMR(X, Omega)
    
    while max_regret >= epsilon:
        print(f"Regret actuel: {max_regret}")
        print(f"Question: préférez-vous {x_star} ou {y_star} ?")
        
        # Poser la question au décideur
        answer = Query(x_star, y_star)
        
        # Mettre à jour Omega en fonction de la réponse
        Omega = Update(Omega, answer, x_star, y_star)
        
        # Recalculer le regret minimax
        max_regret, x_star, y_star = mMR(X, Omega)
    
    return x_star

- Implémente l'algorithme d'élucidation incrémentale pour minimiser le regret minimax.
    Pose des questions au décideur pour affiner ses préférences jusqu'à ce que le regret minimax soit inférieur à 'epsilon'.
    
    Args:
    - X (array-like): Ensemble des solutions disponibles.
    - Omega (list): Polytope des poids admissibles sous forme de contraintes linéaires.
    - epsilon (float): Seuil de tolérance pour le regret minimax.
    
    Returns:
    - array-like: La solution finale 'x_star' recommandée.

In [9]:
# Exemple d'utilisation de l'élucidation
def exemple_elicitation():
    X = np.array([[0.5, 0.2], [0.7, 0.1], [0.6, 0.3]])
    
    # Polytope initial : normalisation des poids
    Omega = [{'coefficients': [1, 1], 'rhs': 1}]  # Initialisation d'un simple polytope

    epsilon = 0.1

    solution = vanilla_incremental_elicitation(X, Omega, epsilon)

    print(f"\nSolution finale recommandée: {solution}")

In [10]:
# Lancer l'exemple
exemple_elicitation()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value -0.1
Coin0511I After Postsolve, objective -0.1, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -0.1 - 0 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value 0.1
Coin0511I After Postsolve, objective 0.1, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.1 - 0 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value -0.2
Coin0511I After Postsolve, objective -0.2, infeasibilities - dual 0 (0), pr